In [1]:
#### TE scraper
#### Credit rating

In [2]:
# modules - libraries:
import requests as r, pandas as pd
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

In [3]:
def get_html(url, parser):
    
    ua = UserAgent()
    HEADERS = {'User-Agent' : str(ua.chrome)}
    
    page = r.get(url, 
                 headers = HEADERS)
    soup = BeautifulSoup(page.content, 
                         f"{parser}")    
    table = soup.find('table', 
                      {'class' : 'table table-hover'})
                         
    return(table)

In [4]:
def df():
    
    table = get_html("https://tradingeconomics.com/country-list/rating", 
                     "html.parser")
    column_headers = [c.text.strip() for c in table.find_all("th")]
    column_headers[0] = "Country"
    cr = pd.DataFrame(columns = column_headers)
    
    for x in table.find_all("tr")[1:]:
        rd = x.find_all("td")
        r = [y.text.strip() for y in rd]
        l = len(cr)
        cr.loc[l] = r
    return([column_headers, cr])

In [5]:
def final_df():
    
    column_headers = (df()[0])
    cr = (df()[1])
    to_drop = column_headers[1:5]
    cr.drop(columns = to_drop, inplace = True)
    cr["TE"] = (cr["TE"].replace("",
                             0,
                             regex = True))
    cr["TE"] = cr["TE"].astype(dtype = int)
    cr["TE"] = (cr["TE"].replace(0,
                                 "",
                                 regex = True))
    cr.sort_values(by = "Country",
                   ascending = True,
                   ignore_index = True,
                   inplace = True)
    
    return(cr)

In [6]:
def save_excel(eng):
    
    column_headers = (df()[0])
    cr = final_df()
    try:
        writer = pd.ExcelWriter("path\\credit_rating.xlsx",
                                engine = eng)
        cr.to_excel(writer,
                    index = False,
                    sheet_name = 'main')
        workbook = writer.book
        worksheet = writer.sheets['main']
        fr = workbook.add_format({'align' : 'center'})
        worksheet.set_column('A:B',
                             None,
                             fr)
        longest = max(column_headers, key = len)
        res = len(longest)
        final_width = res + 20
        worksheet.set_column('A:A',
                             final_width)

        writer.save()
        print("Dataframe saved!")

    except:

        print("Error, dataframe not saved, please check again!")
        
save_excel("xlsxwriter")